In [54]:
import numpy as np
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [90]:
def iplscr(x_d, x_d0, t_d, n=9):
    
    _sum = 0.0
    t_critical = 1.0 / (4.0 * np.pi)

    if t_d <= t_critical:    
        for i in xrange(-n, n):    
            a = x_d - 2*i
            b = 4.0 * t_d      
            _sum = _sum + np.exp(-(a - x_d0) ** 2 / b) + np.exp(-(a + x_d0) ** 2 / b)
            
        val = 1.0 / np.sqrt(4.0 * np.pi * t_d) * _sum
    elif t_d > t_critical:

        for i in xrange(1, n):
            a = i*np.pi
            _sum = _sum + np.exp(-t_d*a ** 2)*np.cos(a*x_d)*np.cos(a*x_d0)

            val = 1.0 + 2.0 * _sum
    
    return val
 

In [91]:
print iplscr(0.0, 0.1, 1.0)

1.00009838335


In [57]:
def ipscr(x_d, y_d, z_d, x_d0, y_d0, z_d0, t_d, x_de, y_de, z_de):
    x  = x_d / x_de
    x0 = x_d0 / x_de
    y  = y_d / y_de
    y0 = y_d0 / y_de
    z  = z_d / z_de
    z0 = z_d0 / z_de
    tx = t_d / (x_de * x_de)
    ty = t_d / (y_de * y_de)
    tz = t_d / (z_de * z_de)

    temp1 = iplscr(x, x0, tx)
    temp2 = iplscr(y, y0, ty)
    temp3 = iplscr(z, z0, tz)

    return temp1 * temp2 * temp3 / (x_de * y_de * z_de)

In [58]:
x_d, y_d, z_d, x_d0, y_d0, z_d0, t_d, x_de, y_de, z_de = 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 1.0, 1.0, 1.0, 1.0
print ipscr(x_d, y_d, z_d, x_d0, y_d0, z_d0, t_d, x_de, y_de, z_de)

1.00029517908


In [163]:
def cpsrc(x_d, y_d, z_d, x_d0, y_d0, z_d0, t_d, t_0, x_de, y_de, z_de):
    a = t_0
    b = t_d
    
    def func(x):
        t_dvar = x
        return ipscr(x_d, y_d, z_d, x_d0, y_d0, z_d0, t_dvar, x_de, y_de, z_de)

    y, err = integrate.quad(func=func, a=a, b=b)
    return y

In [60]:
x_d, y_d, z_d = 0.0, 0.0, 0.0
x_d0, y_d0, z_d0 = 0.1, 0.1, 0.1
t_0, t_d = 0.00000001, 1.0
x_de, y_de, z_de = 1.0, 1.0, 1.0

print cpsrc(x_d, y_d, z_d, x_d0, y_d0, z_d0, t_d, t_0, x_de, y_de, z_de)

3.77731926114


In [171]:
def clsrc(x_d, y_d, z_d, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1, x_de, y_de, z_de, t_0, t_d):
    # Curve Definition
    def well_curve(alpha, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1):
        deltaX = (x_d1 - x_d0)
        deltaY = (y_d1 - y_d0)
        deltaZ = (z_d1 - z_d0)
        x = x_d0 + alpha * deltaX
        y = y_d0 + alpha * deltaY
        z = z_d0 + alpha * deltaZ 
        return x, y, z    
    
    L = np.sqrt( (x_d1-x_d0) ** 2 + (y_d1-y_d0) ** 2 + (z_d1-z_d0) ** 2 )
    X = np.sqrt( (x_d-x_d0) ** 2  + (y_d-y_d0) ** 2  + (z_d-z_d0) ** 2  )
    cos_beta = ((x_d-x_d0)*(x_d1-x_d0) + (y_d-y_d0)*(y_d1-y_d0) + (z_d-z_d0)*(z_d1-z_d0)) / (L * X)

    I = 1.0 / (4.0 * np.pi * L) * np.log( (1.0 - (X / L) * cos_beta \
        + np.sqrt((X / L) ** 2 - 2.0 * (X / L) * cos_beta + 1.0)) / ((X / L) * (1.0 - cos_beta)))

    # Integrate along X
    def func(x):
        alpha = x
        xe, ye, ze = well_curve(alpha, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1)
        l = np.sqrt((xe - x_d0)**2 + (ye - y_d0)**2 + (ze - z_d0)**2)
        r_d = np.sqrt(X**2 + l**2 - 2.0*X*l*cos_beta)

        return cpsrc(x_d, y_d, z_d, xe, ye, ze, t_d, t_0, x_de, y_de, z_de) - 1.0/(4.0 * np.pi * r_d)

    #y = integrate.romberg(function=func, a=0.0, b=1.0)
    #y, err = integrate.fixed_quad(func=func, a=0.0, b=1.0)
    y, err = integrate.quad(func=func, a=0.0, b=1.0)
    return y / L + I

In [108]:
x_d, y_d, z_d = 0.0, 0.0, 0.0
x_d0, y_d0, z_d0 = 0.1, 0.1, 0.1
x_d1, y_d1, z_d1 = 0.5, 0.5, 0.5
t_0, t_d = 0.00000001, 1.0
x_de, y_de, z_de = 1.0, 1.0, 1.0
r_d = 0.15

#print clsrc(x_d, y_d, z_d, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1, x_de, y_de, z_de, t_0, t_d, r_d)

In [161]:
# Parameters Definition
x_e, y_e, z_e = 900., 900.0, 300.0

k_x, k_y, k_z = 2000., 2000., 500.
c_t = 7.7e-5
B_o = 1.0
phi = 0.2
mu  = 1.0
P_i = 6000.

x_0, y_0, z_0 = 400., 450., 150.
x_1, y_1, z_1 = 500., 450., 150.

r_w = 0.125
t_0, t_d = 0.000000000001, 1.0

r_dw = r_w / x_e
x_d0, y_d0, z_d0 = x_0 / x_e, y_0 / x_e, z_0 / x_e
x_d1, y_d1, z_d1 = x_1 / x_e, y_1 / x_e, z_1 / x_e


x_de, y_de, z_de = x_e / x_e, y_e / x_e, z_e / x_e 
x_d, y_d, z_d = 0.0, 0.0, z_d0

In [70]:
# Parameters Definition
x_e, y_e, z_e = 900., 900.0, 300.0

k_x, k_y, k_z = 2000., 2000., 500.
c_t = 7.7e-5
B_o = 1.0
phi = 0.2
mu  = 1.0
P_i = 6000.

x_0, y_0, z_0 = 400., 450., 150.
x_1, y_1, z_1 = 500., 450., 150.

r_w = 0.125
t_0, t_d = 0.000000000001, 1.0

r_dw = r_w / x_e
x_d0, y_d0, z_d0 = x_0 / x_e, y_0 / x_e, z_0 / x_e
x_d1, y_d1, z_d1 = x_1 / x_e, y_1 / x_e, z_1 / x_e


x_de, y_de, z_de = x_e / x_e, y_e / x_e, z_e / x_e 


n_points = 2

t = 10.0

x_points = np.linspace(0.0, x_de, num=n_points)
y_points = np.linspace(0.0, y_de, num=n_points)

x_d, y_d, z_d = 0.0, 0.0, z_d0


P_d = np.zeros([n_points, n_points])

for i, x in enumerate(x_points):
    for j, y in enumerate(y_points):
        P_d[i,j] = clsrc(x, y, z_d, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1, x_de, y_de, z_de, t_0, t_d)

fig = plt.figure()

X, Y = np.meshgrid(x_points, y_points)
plt.contourf(X, Y, P_d)

plt.show()

In [82]:
print iplscr(x_d=0.5, x_d0=x_d0, t_d=0.01)

2.61146789706


In [72]:
clsrc(0.0, 0.0, z_d, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1, x_de, y_de, z_de, t_0, t_d)

21.213179988388898

In [83]:
ipscr(0.5, 0.5, z_d, x_d0, y_d0, z_d0, 0.01, x_de, y_de, z_de)

23.366252983747682

In [119]:
cpsrc(0.5, 0.5, z_d, x_d0, y_d0, z_d0, 0.01, t_0, x_de, y_de, z_de)

0.19411408781926248

In [170]:
clsrc(0.0, 0.0, z_d, x_d0, y_d0, z_d0, x_d1, y_d1, z_d1, x_de, y_de, z_de, t_0, 0.01)

0.707106781187
0.669941439335
0.746346681269
0.673981156155
0.741868418728
0.680940311039
0.734281730149
0.69029139149
0.724332278003
0.701282831293
0.712979103962
0.669137791111
0.747244241974
0.671561562667
0.74454399791
0.677128462299
0.738417551005
0.685356236036
0.729548998212
0.695638075574
0.718764669601


-0.90077313799981074

In [127]:
r_dw

0.0001388888888888889

In [153]:
r_d

0.15

In [141]:
z_d0

0.16666666666666666

In [142]:
z_d

0.0

In [148]:
r_d

0.15